#### Sociology 128D: Mining Culture Through Text Data: Introduction to Social Data Science – Summer '22

# Notebook 4:  Visualizing Sentiment and More using the Empath Library

## Sentiment Analysis and Dictionary Methods using `Empath`

Sentiment analysis is a broad topic, and we will revisit it later. In this notebook, we will focus on dictionary methods, which offer one approach to sentiment analysis as well as offering analyses beyond sentiment. Dictionary methods are among the most straightforward of methods for computational text analysis. A "dictionary" or "lexicon" in this sense is a list of words that we have decided in advance are related to a given topic, or "lexical category," like positive sentiment. To use these dictionaries, we count the words in a given document that are in that dictionary, often taking into account that longer documents aren't necessarily more positive or negative in a meaningful way by virtue of having more words. If we have a list of positive words and a list of negative words, and more words from the first list are in a document, we might say the document is positive. (Sentiment analysis is not restricted to dictionary methods, but that's where we'll start.)

In this notebook, we will learn to use dictionary methods for sentiment analysis and for more general exploration of topics in social media posts using `pandas`, `seaborn`, and Python's [`Empath` library](https://github.com/Ejhfast/empath-client). You can read more about this library in the original paper [here](https://hci.stanford.edu/publications/2016/ethan/empath-chi-2016.pdf). Researchers commonly use paid sets of dictionaries like those available from [LIWC](https://liwc.wpengine.com/). However, the original paper behind `Empath` suggests that similar categories are correlated with LIWC. `Empath` offers 194 lexical categories in total—and it's free! Another benefit is the ease of creating custom dictionaries. To do this, you simply provide a set of "seed words" (i.e., words to "seed" or begin your category), and `Empath` will identify similar words to fill out the dictionary for you. (We'll talk more about how that works later.)

<div class="alert alert-info">
We will consider a few ways to use dictionary methods to analyze social processes. To do this, we're going to use a corpus of posts to Reddit's r/jobs message board between January 1 and December 31, 2020. Specifically, we will consider relationships like the following:
<ul>
    <li>how and when people interact with social media (time of day, day of week)</li>
    <li>how time relates to content and engagement (e.g., the number of replies or upvotes)
    <li>the relationship between content and engagement</li>
    <li>relationships among different types of content (specifically, lexical categories, including sentiment)</li>
</ul>
</div>

More generally, we will consider how these relate to **social processes** and **social institutions**. The concept of a *social insitution* has been defined many ways. Consider, for instance, Granovetter's characterization of social institutions in *Society and Economy* (2017: p. 171): "patterns of ideas or norms that influence but only incompletely determine how actors approach the problems they mean to solve."

The standard work day and standard work week are norms that exert considerable influence on the organization of social life. It makes sense that people working a standard schedule might feel a bit buoyed on the weekend. But what about people who work different hours, or who aren't working at all during a given period of time? [Young and Lim (2014)](https://sociologicalscience.com/time-network-good/) find that both working and unemployed individuals experience similar changes in subjective wellbeing throughout the course of the week. Despite the fact that unemployed individuals are not working during the work week, *both* groups report greater wellbeing on weekends. Young and Lim argue that how people experience the rhythm of the week depends on the availability of other people as much as it does on just having time to do things (roughly speaking). That extends to the opportunities that exist based on formal and informal sanctions. Think about how opportunties are shaped by the day of the week. Places of business are often open later on weekends, for example. Now consider how our time-based expectations become entrenched and moralized. To take one example, people who do shift work often get judged for having a drink after work when "after work" is early in the morning, which is generally seen as an inappropriate time to drink.

We might also use these data to ask questions like whether users writing about a certain topic also tend to write about another topic or set of topics, and we can come up with hypotheses for why we might observe certain associations.

First, we will import the libraries we need for the analyses we'll run. You may need to install `Empath` and download the language model for spaCy. You can run the following lines to do so. The `!` tells Jupyter you want to execute commands as if you were using the command line (or terminal).

In [ ]:
!pip install --user empath
!python -m spacy download en_core_web_sm

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
import spacy
import time
import warnings

from empath import Empath
from scipy.stats import pearsonr

%matplotlib inline

warnings.filterwarnings("ignore") # comment out this line to see warnings

sns.set_theme(style="darkgrid")

`pandas` has a built-in method for creating a correlation matrix (i.e., a variable-by-variable table of correlation coefficients), but it does not immediately tell us whether the result is significant. We have imported the `pearsonr` function from the `scipy.stats` module. Below, we define a helper function using simple [*if-else* logic](https://cs.stanford.edu/people/nick/py/python-if.html) to give us a correlation coefficient either with a p-value for results that are statistically non-significant or with the conventional number of asterisks (stars) indicating the level of statistical significance.

In [2]:
def quick_r(x: str, y: str, df: pd.core.frame.DataFrame) -> str:
    """
    Returns (as a str) the Pearson's correlation coefficient
    with the p-value or asterisks for significant correlations
    *p<0.05, **p<0.01, ***p<0.001
    """
    tmp = df[[x, y]].dropna()
    r, pval = pearsonr(tmp[x], tmp[y])
    if pval < 0.001:
        r = f"{r:.2f}***"
    elif pval < 0.01:
        r = f"{r:.2f}**"
    elif pval < 0.05:
        r = f"{r:.2f}*"
    else:
        r = f"{r:.2f} (p = {pval:.2f})"
    return r

Next, we will load the corpus. You can find the corpus in Canvas in the Data/ directory.

[r/jobs](https://old.reddit.com/r/jobs/) provides the following description:

<blockquote>We aim to empower job seekers and employees through the promotion of their best interests, advice and encouragement. </blockquote>

#### Where is the corpus from?

I pulled ~150 posts to Reddit's r/jobs community per day from [pushshift.io](https://pushshift.io/)'s API for each day in 2020. Many thanks for the work they do to make data available! I have cleaned this corpus a bit already, although it is far from perfect and we will still stuff like markdown containing links. It includes 49,872 posts in total. The metadata/variables associated wih the text include the submission's score ("karma" or "upvotes" in Reddit parlance), the number of replies the post received, the title of the submission, the text of the submission, and various markers of time. Specifically, I have included the date and separate variables for the day of the month (1-31), the day of the week (as integers 0-6 and strings "Sunday" to "Monday"), day of the year (1-366, as it was a leap year!), week of the year, and the hour of the day (0-23). It is important to note that Reddit attracts an international audience, so the time of day (and, to an extent, the day itself) may be relative to Pushshift, not to each individual user. Although Reddit allows for relative anonymity through the use of usernames/handles, I have provided an ID field ("author_id") in place of the username of the author.

In [3]:
df = pd.read_json("data/rjobs_preprocessed_v3.json")

Let's take a quick look at the data. We'll get to an example post and some analyses of the content, but it's important to first have a sense of what the dataset is like overall.

In [ ]:
df.head()

`.shape` provides the number of rows (posts) first, followed by the number of columns (variables)

In [ ]:
df.shape

In [ ]:
print(f"{df.shape[0]:,} rows and {df.shape[1]} columns.")

In [ ]:
df.columns

## Sample Posts

In [ ]:
sample = df.sample(1)
for _, row in sample.iterrows():
    print(row["text"], "\n\n")

### First, let's examine whether there seem to be general trends over the course of the week in the number of posts, the number of comments on posts, and how well posts do ("karma" or "upvotes").

It may seem a bit odd to look for evidence of a relationship between day of the week and other variables, especially since we could consider different starting and ending points for the week (e.g., saying the week starts on Sunday vs. Monday). However, we may have good reasons to expect that people behave differently on Mondays than they do on weekends, and that may be reflected in outcomes such as how often they post, what they post about, how successful their posts are, and how many replies their posts get.

Let's start by removing the time from the date.

In [ ]:
df["date"] = df["date"].dt.date

In [ ]:
daysofweek = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

sns.lineplot(x="dayofweek", y="posts_that_day", ci=False,
             data=df[["dayofweek", "date", "posts_that_day"]].groupby("date").mean())
plt.title("Posts to r/jobs in 2020, by Day of Week")
plt.ylabel("Posts")
plt.ylim(0, 250)
plt.xticks(ticks=range(0,7), labels=daysofweek, rotation=30)
plt.xlabel("Day of Week")
plt.show()

sns.lineplot(x="dayofweek", y="num_comments", data=df, ci=False)
plt.title("Posts to r/jobs in 2020, by Day of Week")
plt.ylabel("Comments on Posts")
plt.ylim(0, 15)
plt.xticks(ticks=range(0,7), labels=daysofweek, rotation=30)
plt.xlabel(f"Day of Week\nr = {quick_r('dayofweek', 'num_comments', df)}")
plt.show()


sns.lineplot(x="dayofweek", y="score", data=df, ci=False)
plt.title("Posts to r/jobs in 2020, by Day of Week")
plt.ylabel("Karma")
plt.ylim(0, 5)
plt.xticks(ticks=range(0,7), labels=daysofweek, rotation=30)
plt.xlabel(f"Day of Week")
plt.show()

### Now let's examine whether there is substantial variation over the course of the day.

In [ ]:
hours = ["Midnight"] + [f"{i} AM" for i in range(1,12)] + ["Noon"] + [f"{i-12} PM" for i in range(13,24)]

df["hour"].value_counts().loc[list(range(24))].plot(kind="line")
plt.xticks(ticks=list(range(24)), labels=hours, rotation=60)
plt.title("Posts to r/jobs in 2020 by Time of Day")
plt.ylabel("Count")
plt.xlabel("Time of Day")
plt.show()

In [ ]:
hours = ["Midnight"] + [f"{i} AM" for i in range(1,12)] + ["Noon"] + [f"{i-12} PM" for i in range(13,24)]

sns.lineplot(x="hour", y="num_comments", data=df, ci=False)
plt.title("Average Replies to Posts to r/jobs by Time of Day")
plt.xticks(ticks=range(0,24,3), labels=hours[::3])
plt.xlabel("Time When Post is Made")
plt.ylabel(f"Average Comments")
plt.ylim(0, 15)
plt.show()

In [ ]:
hours = ["Midnight"] + [f"{i} AM" for i in range(1,12)] + ["Noon"] + [f"{i-12} PM" for i in range(13,24)]

sns.lineplot(x="hour", y="score", data=df, ci=False)
plt.title("Karma of Posts to r/jobs by Time of Day")
plt.xticks(ticks=range(0,24,3), labels=hours[::3])
plt.xlabel("Time When Post is Made")
plt.ylabel("Karma")
plt.ylim(0, 5)
plt.show()

### How about variation over the course of the year?

In [ ]:
df["dayofyear"].value_counts().loc[list(range(1,367))].plot(kind="line")
plt.title("Posts r/jobs in 2020")
plt.ylabel("Count")
plt.xlabel("Day of Year")
plt.show()

In [ ]:
sns.lineplot(x="dayofyear", y="num_comments", data=df, ci=False)
plt.title("Average Replies to Posts to r/jobs over the Course of 2020")
plt.xlabel("Day of Year")
plt.ylabel("Average Comments")
plt.show()

In [ ]:
sns.lineplot(x="dayofyear", y="score", data=df, ci=False)
plt.title("Karma of Posts to r/jobs over the Course of 2020")
plt.xlabel("Day of Year")
plt.ylabel("Karma")
plt.show()

In [ ]:
sns.lineplot(x="week", y="score", data=df, ci=False)
plt.title("Karma of Posts to r/jobs by Week")
plt.xlabel("Week")
plt.ylabel("Karma")
plt.show()

We see quite a bit variation with respect to time in these graphs. Based on these data, it looks like users most frequently post to r/jobs early in the week but take a break on the weekend, yet the average number of replies is higher on the weekend (perhaps due to the lack of new posts). Most posts are made in the afternoon of evening, peaking around 7 PM. The posts that earn the most karma and get the most replies are made around noon. There are days over the course of 2020 when there is a lot more activity. That could be an issue with the dataset, or there could be something sociologically interesting about those days.

Let's take a quick look at the distribution of scores and then dive into sentiment.

In [ ]:
sns.kdeplot("score", data = df)
plt.title("Distribution of Karma of Posts to r/jobs in 2020")
plt.xlabel("Karma")
plt.show()

This distribution highly skewed. Let's take a look at an outlier. We'll look for the post with the highest score.

In [ ]:
print(df[df.score==df.score.max()]["text"].values[0])

Let's zoom in and see what the distribution looks like without some of the posts in the long right tail.

In [ ]:
sns.kdeplot("score", data = df[df["score"] <= 10])
plt.title("Distribution of Karma of Posts to r/jobs in 2020"
         "\n(among posts with <= 10 karma)")
plt.xlabel("Karma")
plt.show()

It seems like the data are clustered pretty tightly around 1 karma, which is where a post starts. Let's see how many posts have something other than 1 karma and look at the distribution of replies.

In [ ]:
df[df["score"]!=1].shape

In [ ]:
sns.kdeplot("num_comments", data = df[df["num_comments"] <= 50])
plt.title("Distribution of the Number of Replies to Posts to r/jobs in 2020"
         "\n(among posts with <= 50 replies)")
plt.xlabel("Replies")
plt.show()

We also know that people write posts of different lengths. The length of a post could be related to the score, to the topics someone discusses, or other factors.

In [ ]:
df["word_count"] = df["text"].apply(lambda x: len(x.split()))

sns.kdeplot(df["word_count"])
plt.title("Distribution of Word Counts in Posts to r/jobs 2020")
plt.xlabel("Word Count")
plt.show()

sns.kdeplot(df[df.word_count < 400]["word_count"])
plt.title("Distribution of Word Counts in Posts to r/jobs 2020"
         "\n(zoomed in to < 400 words to exclude outliers)")
plt.xlabel("Word Count")
plt.show()

In [ ]:
df[["score", "num_comments", "word_count"]].describe()

In [ ]:
df[["score", "num_comments", "word_count"]].corr()

In [ ]:
print(df[df["word_count"]<75].sample(1)["text"].values[0])

In [ ]:
print(df[df["word_count"]>215].sample(1)["text"].values[0])

## Using the `Empath` library

Now let's move on to analyzing posts using dictionary methods via the `Empath` library. We'll create the object `lexicon` that will analyze posts using 194 separate dictionaries, which range from positive or negative emotion to topics as diverse as optimism, terrorism, fashion, and eating.

In [ ]:
lexicon = Empath()

In [ ]:
# Number of categories
len(lexicon.cats.keys())

<h1>Lexical categories Provided by <tt>Empath</tt></h1>
<table class='table table-striped'> <tr><th>achievement</th><th>affection</th><th>aggression</th><th>air_travel</th></tr> <tr><th>alcohol</th><th>ancient</th><th>anger</th><th>animal</th></tr> <tr><th>anonymity</th><th>anticipation</th><th>appearance</th><th>art</th></tr> <tr><th>attractive</th><th>banking</th><th>beach</th><th>beauty</th></tr> <tr><th>blue_collar_job</th><th>body</th><th>breaking</th><th>business</th></tr> <tr><th>car</th><th>celebration</th><th>cheerfulness</th><th>childish</th></tr> <tr><th>children</th><th>cleaning</th><th>clothing</th><th>cold</th></tr> <tr><th>college</th><th>communication</th><th>competing</th><th>computer</th></tr> <tr><th>confusion</th><th>contentment</th><th>cooking</th><th>crime</th></tr> <tr><th>dance</th><th>death</th><th>deception</th><th>disappointment</th></tr> <tr><th>disgust</th><th>dispute</th><th>divine</th><th>domestic_work</th></tr> <tr><th>dominant_heirarchical</th><th>dominant_personality</th><th>driving</th><th>eating</th></tr> <tr><th>economics</th><th>emotional</th><th>envy</th><th>exasperation</th></tr> <tr><th>exercise</th><th>exotic</th><th>fabric</th><th>family</th></tr> <tr><th>farming</th><th>fashion</th><th>fear</th><th>feminine</th></tr> <tr><th>fight</th><th>fire</th><th>friends</th><th>fun</th></tr> <tr><th>furniture</th><th>gain</th><th>giving</th><th>government</th></tr> <tr><th>hate</th><th>healing</th><th>health</th><th>hearing</th></tr> <tr><th>help</th><th>heroic</th><th>hiking</th><th>hipster</th></tr> <tr><th>home</th><th>horror</th><th>hygiene</th><th>independence</th></tr> <tr><th>injury</th><th>internet</th><th>irritability</th><th>journalism</th></tr> <tr><th>joy</th><th>kill</th><th>law</th><th>leader</th></tr> <tr><th>legend</th><th>leisure</th><th>liquid</th><th>listen</th></tr> <tr><th>love</th><th>lust</th><th>magic</th><th>masculine</th></tr> <tr><th>medical_emergency</th><th>medieval</th><th>meeting</th><th>messaging</th></tr> <tr><th>military</th><th>money</th><th>monster</th><th>morning</th></tr> <tr><th>movement</th><th>music</th><th>musical</th><th>negative_emotion</th></tr> <tr><th>neglect</th><th>negotiate</th><th>nervousness</th><th>night</th></tr> <tr><th>noise</th><th>occupation</th><th>ocean</th><th>office</th></tr> <tr><th>optimism</th><th>order</th><th>pain</th><th>party</th></tr> <tr><th>payment</th><th>pet</th><th>philosophy</th><th>phone</th></tr> <tr><th>plant</th><th>play</th><th>politeness</th><th>politics</th></tr> <tr><th>poor</th><th>positive_emotion</th><th>power</th><th>pride</th></tr> <tr><th>prison</th><th>programming</th><th>rage</th><th>reading</th></tr> <tr><th>real_estate</th><th>religion</th><th>restaurant</th><th>ridicule</th></tr> <tr><th>royalty</th><th>rural</th><th>sadness</th><th>sailing</th></tr> <tr><th>school</th><th>science</th><th>sexual</th><th>shame</th></tr> <tr><th>shape_and_size</th><th>ship</th><th>shopping</th><th>sleep</th></tr> <tr><th>smell</th><th>social_media</th><th>sound</th><th>speaking</th></tr> <tr><th>sports</th><th>stealing</th><th>strength</th><th>suffering</th></tr> <tr><th>superhero</th><th>surprise</th><th>swearing_terms</th><th>swimming</th></tr> <tr><th>sympathy</th><th>technology</th><th>terrorism</th><th>timidity</th></tr> <tr><th>tool</th><th>torment</th><th>tourism</th><th>toy</th></tr> <tr><th>traveling</th><th>trust</th><th>ugliness</th><th>urban</th></tr> <tr><th>vacation</th><th>valuable</th><th>vehicle</th><th>violence</th></tr> <tr><th>war</th><th>warmth</th><th>water</th><th>weakness</th></tr> <tr><th>wealthy</th><th>weapon</th><th>weather</th><th>wedding</th></tr> <tr><th>white_collar_job</th><th>work</th><th>worship</th><th>writing</th></tr> </table>

In [ ]:
df["text"] = df["text"].apply(str.lower) # lowercasing the text

In [ ]:
example = df.sample(1)
example_text = example["text"].values[0]
print(example_text)

In [ ]:
lexicon.analyze(example_text)

The output is a dictionary: each lexical category is a key and the score is the value. Let's define a function to print the output in a more manageable way. The function below finds all lexical categories where the score is nonzero for a given post, sorts them, and then print at most <texttt>max_results</texttt> results. 

In [ ]:
def analyze_post(post: str, normalize:bool=True, max_results=10):
    """
    Prints dictionaries and scores for nonzero scores
    """
    analysis = lexicon.analyze(post, normalize=normalize)
    nonzero = []
    for key, value in analysis.items():
        if value > 0.0:
            nonzero.append((key, value))
    nonzero = sorted(nonzero, key=lambda x: x[1], reverse=True)
    nonzero = nonzero[:min(max_results, len(nonzero))]
    for lex in nonzero:
        print(f"{lex[1]:.2f} {lex[0]}")

**Normalizing** is an important choice when using these methods. Compare the results below. Whether normalizing ultimately matters may depend on what we're trying to accomplish. For now, let's stick to normalizing. Posts vary a lot in length.

In [ ]:
analyze_post(example_text, normalize=False)

In [ ]:
analyze_post(example_text, normalize=True)

**Preprocessing** is also an important step for a lot of tasks. The code below loads a language model from `spacy`, disables some of its functionality (Named Entity Recognition) to make it run faster, and then preprocesses the sentence using the model. `nlp(example_text)` analyzes the string using the language model we have loaded, and it does things like part-of-speech tagging. We can also remove stopwords and [lemmatize](https://en.wikipedia.org/wiki/Lemmatisation).

In [ ]:
nlp = spacy.load("en_core_web_sm", disable=["ner"])

In [ ]:
prep_example_text = nlp(example_text)
prep_example_text = [word.lemma_ for word in prep_example_text if not word.is_stop]
print(prep_example_text)

In [ ]:
analyze_post(prep_example_text, normalize=True)

In [ ]:
lexicon.analyze(prep_example_text, normalize=True)["negative_emotion"]

The dataset has a variable ("preprocessed") with preprocessed versions of the posts and variables for each of the dictionaries provided by `Empath`.

### Most and least positive and negative posts

Let's take a look at the most positive post according to the positive_sentiment lexicon:

In [ ]:
most_pos = df[df.positive_emotion==df.positive_emotion.max()]

display(most_pos[["score", "num_comments", "date"]])

print(most_pos["title"].values[0])
print(most_pos["selftext"].values[0])

And the most negative post:

In [ ]:
most_pos = df[df.negative_emotion==df.negative_emotion.max()]

display(most_pos[["score", "num_comments", "date"]])

print(most_pos["title"].values[0])
print(most_pos["selftext"].values[0])

What about the *least positive* or *least negative*? Unfortunately, these are going to be 0.0 and apply to a large portion of posts. Even if we standardize these, the lowest values will still be the posts that don't use relevant words, not necessarily words meaning the *opposite*. In fact, we'll often see that posts that use words related to one category use words related to the opposite. Positive and negative sentiment words are actually highly correlated according to this dictionary-based approach.

In [ ]:
print(f"Minimum for positive and negative emotion: {df.positive_emotion.min()} and {df.negative_emotion.min()}")

In [ ]:
print(f"Correlation between positive and negative emotion: r = {quick_r('negative_emotion', 'positive_emotion', df)}")

In [ ]:
sns.scatterplot(x="negative_emotion", y="positive_emotion", data=df)
plt.xlabel("Negative Emotion Words")
plt.ylabel("Positive Emotion Words")
plt.title("Relationship between Negative Emotion Words\nand Positive Emotion Words in r/jobs in 2020")
plt.show()

### Temporal Trends in Sentiment

Do posts tend to relate to positive or negative sentiment to different degrees at different points in time? We'll plot these to find out and even check some of the correlations. Importantly, days and weeks are cyclical, so establishing a linear trend (as with a correlation coefficient) is unlikely. We might also expect seasonal effects over the course of the year if people are shorter on money at different times, if having money seems more important at different times (e.g., around specific holidays), or if people tend to lose their jobs more at certain times of year.

*On the other hand*, we're looking at 2020. There may be shifts in how much emotion words are used, on average, as the pandemic worsens. As a consequence, we may see linear trends over the course of the year.

In [ ]:
sns.lineplot(x="hour", y="positive_emotion", data=df, ci=False)
plt.title("Positive Emotion over the Course of the Day")
plt.ylabel("Positive Emotion Words")
plt.xlabel("Time of Day")
plt.xticks(ticks=range(0,24,3), labels=hours[::3])
plt.show()

sns.lineplot(x="hour", y="negative_emotion", data=df, ci=False)
plt.title("Negative Emotion over the Course of the Day")
plt.ylabel("Negative Emotion Words")
plt.xlabel("Time of Day")
plt.xticks(ticks=range(0,24,3), labels=hours[::3])
plt.show()

In [ ]:
plt.title("Posts to r/jobs in 2020, by Day of Week")
sns.lineplot(x="dayofweek", y="positive_emotion", data=df, ci=False)
plt.ylabel("Positive Emotion Words")
plt.xticks(ticks=range(0,7), labels=daysofweek, rotation=30)
plt.xlabel(f"Day of Week\nr = {quick_r('dayofweek', 'positive_emotion', df)}")
plt.show()

plt.title("Posts to r/jobs in 2020, by Day of Week")
sns.lineplot(x="dayofweek", y="negative_emotion", data=df, ci=False)
plt.ylabel("Negative Emotion Words")
plt.xticks(ticks=range(0,7), labels=daysofweek, rotation=30)
plt.xlabel(f"Day of Week\nr = {quick_r('dayofweek', 'negative_emotion', df)}")
plt.show()

In [ ]:
plt.title("Posts to r/jobs in 2020")
sns.lineplot(x="week", y="positive_emotion", data=df, ci=False)
plt.ylabel("Positive Emotion Words")
plt.xlabel(f"Week of 2020\nr = {quick_r('week', 'positive_emotion', df)}")
plt.show()

plt.title("Posts to r/jobs in 2020")
sns.lineplot(x="week", y="negative_emotion", data=df, ci=False)
plt.ylabel("Negative Emotion Words")
plt.xlabel(f"Week of 2020\nr = {quick_r('week', 'negative_emotion', df)}")
plt.show()

### Standardizing Variables

Before we dive into the rest of our analyses, we are going to "standardize" the variables. If you have some background in statistics, this may be familiar: we subtract the mean of a variable from each value and divide by the standard deviation, which is the same as calculating a [z-score](https://en.wikipedia.org/wiki/Z-score). This sets the mean of the variable to zero and the standard deviation to one.

Standardizing variables offers two major advantages that are relevant to this class. First, sometimes variables are just difficult to interpret. Have you ever been asked whether you agree or disagree with a statement on a scale of 1-7 (or 1-5, or 0-100)? What would it mean to say that the difference between two groups is 0.24 on that 1-7 scale? Often, psychologists and others standardize variables so that differences have a more straightforward interpretation. This interpretation is that a difference is some fraction of a standard deviation. Put differently, we can discuss the magnitude of a difference with respect to how much variance is in the data. Is the difference large, considering how much the data vary in general? When analyzing the results of an experiment, a psychologist might say that the treatment has an effect on the outcome of 0.4 standard deviations. That's pretty opaque—but many people find it more intuitive than saying the effect is 0.24 out of seven.

The second and most immediately obvious advantage is that we can plot standardized variables at the same time. Examine the two plots below. The first plot includes unstandardized variables. If we were to look at any one of these variables, we may see more ups and downs, but the additional variables change the scale of the y-axis. This means we have zoomed out, so the ups and downs are more difficult to see. (Whether the ups and downs matter is another question, and we'll talk more about this!)

In [ ]:
daysofweek = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

df[["dayofweek", "money", "work", "love", "nervousness", "alcohol"]].groupby("dayofweek").mean(["money", "work", "love", "nervousness", "alcohol"]).plot(kind="line")
plt.xticks(ticks=range(0,7), labels=daysofweek, rotation=30)
plt.show()

In [ ]:
df[["dayofweek", "work"]].groupby("dayofweek").mean(["work"]).plot(kind="line")
plt.xticks(ticks=range(0,7), labels=daysofweek, rotation=30)
plt.show()

In [ ]:
df[["money", "work", "love", "nervousness", "alcohol"]].describe()

In [ ]:
def standardize(series):
    """
    Mean-centers a variable and divides by the standard deviation,
    resulting in a mean of zero and a standard deviation of one.
    This puts variables on the same scale and can make comparisons
    a bit easier to see.
    """
    if max(series) == 0.0:
        return series
    mu = np.mean(series.dropna())
    std = np.std(series.dropna(), ddof=1) + 1e-7
    series = [(val-mu)/std for val in series.values]
    return series

The cell below iterates through each lexical category (i.e., dictionary) and standardizes the corresponding variable I have already added to the dataset.

In [ ]:
%%time

for cat in lexicon.cats.keys():
    df[cat] = standardize(df[cat])

Contrast this to the similar table above. Here, the means (i.e., averages) are effectively zero because we *mean centered* the data by subtracting the mean of each variable. The standard deviations are effectively 1 because we divided each datapoint by the standard deviation for the variable. The values aren't *exactly* 0 or 1, but they are very, very close!

In [ ]:
df[["money", "work", "love", "nervousness", "alcohol"]].describe()

Now if we plot the variables we looked at above over the course of the week, this time using the standardized versions, we don't have the problem of "zooming out" and missing the trends. Looking at this figure, it seems people post more about money on the weekends, for example.

In [ ]:
daysofweek = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

df[["dayofweek", "money", "work", "love", "nervousness", "alcohol"]].groupby("dayofweek").mean(["money", "work", "love", "nervousness", "alcohol"]).plot(kind="line")
plt.xticks(ticks=range(0,7), labels=daysofweek, rotation=30)
plt.xlabel("Day of the Week")
plt.show()

This plot is a mess, but we see a lot of change from day to day!

### Do people talk about different things at different times?

In [ ]:
daysofweek = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

vars_to_plot = ["party", "religion", "love", "money"]

plot_ci=None

for var in vars_to_plot:
    sns.lineplot(x="dayofweek", y=var, data=df, ci=plot_ci)
plt.xticks(ticks=range(0,7), labels=daysofweek, rotation=30)
plt.xlabel("Day of the Week")
plt.legend(labels=[v.capitalize() for v in vars_to_plot])
plt.show()

In [ ]:
df[["party", "religion", "love", "money"]].corr()

In [ ]:
vars_to_plot = ["shame", "body", "eating"]

plot_ci=None

for var in vars_to_plot:
    sns.lineplot(x="hour", y=var, data=df, ci=plot_ci)
plt.xticks(ticks=range(0,24,3), labels=hours[::3], rotation=30)
plt.xlabel("Time of Day")
plt.legend(labels=[v.capitalize() for v in vars_to_plot])
plt.show()

In [ ]:
print(df[df.body==df.body.max()].text.tolist()[0])

In [ ]:
print(df[df.shame==df.shame.max()].text.tolist()[0])

In [ ]:
print(df[df.eating==df.eating.max()].text.tolist()[0])

## Exercises

First, pick a social problem you would like to explore using this corpus. In Exercise 1, you will create a dictionary for that topic in only three lines of code. You will use your dictionary for the remaining exercises.

`Empath` makes creating dictionaries for different lexical categories fairly straightforward on the surface. Beneath the surface, it uses some pretty powerful tools. We'll learn about these tools (such as word embeddings) later in the class. For now, all we need to know is that we can provide a small list of words related to our chosen topic and `Empath` will give us a longer list of related words. We can use that list as our lexicon.

For example, let's say we want to create a lexicon related to stigma. We can seed (start) our lexicon using a handful of words. Let's try these:

In [ ]:
stigma_words = ["stigma", "taboo", "avoid", "aversion", "stereotype", "trash"]

Now we can create our lexicon using the following line of code. The first part (<tt>"stigma"</tt>) is what we are naming the category. The second part (<tt>stigma_words</tt>) is the list we created in the line above). The <tt>model="reddit"</tt> argument means we will find words that are related based on how language is (was) used in Reddit data when `Empath` was created. The alternatives are based on the New York Times and a collection of fiction.

Note: we will talk a LOT more about word embeddings and vector space models in the future, and you will learn how they are used to generate collections of words like the following. The TL;DR is that these tools assume words are "similar" if they are used in similar contexts. This assumption can make it easy to find synonyms, but sometimes tools based on this assumption will think _antonyms_ are highly "similar" in this specific sense. These tools can also reflect social prejudices, which can make them useful for studying those issues (as we will later in the course) but may be a bit of a surprise. **Debiasing** word embeddings and other tools is also an active area of research, and we will discuss that more.

In other words, you may see examples of prejudice inherent to the language used on Reddit (or in fiction or the New York Times), but that doesn't mean that a word that shows up should be seen as part of the reality of your topic.

In [ ]:
lexicon.create_category("stigma", stigma_words, model="reddit", size=100)

Also notice the [n-grams](https://en.wikipedia.org/wiki/N-gram) connected by underscores (for example, "cultural_norm") and compound words (for example, "slut_shaming"). The corpus of posts from r/jobs only has unigrams and has no punctuation. We'll cover those issues more later on in the course. For now, that just means there are words in the lexicon that won't appear in any of the documents in this corpus.

That aside, this custom lexicon can now be used just like those we've already used. The line of code below will add a variable to our dataframe. Be sure to include the <tt>normalize=True</tt> argument when you create your own in Exercise 1! Alternatively, if you'd prefer to use the raw counts, you should use the raw counts for the other lexical categories as well, as we initially did toward the beginning of this notebook. If you opt to use raw counts, please provide your rationale.

In [ ]:
def score_category(s, category, normalize=True):
    """
    This function returns the score for a given document for a particular category
    using Empath's dictionaries. The optional argument `normalize` defaults to False.
    """
    res = lexicon.analyze(s, categories=[category], normalize=normalize)
    if res:
        return res[category]
    return 0.0


df["stigma"] = [score_category(s, "stigma", normalize=True) for s in df["preprocessed"].values]
df["stigma"] = standardize(df["stigma"])

<div class="alert alert-warning"><strong></strong>
There you have it! We've created our own lexicon for the lexical category "stigma" in only three lines of code using Empath!

Now you will do the same.

### Exercise 1
1. Create your custom lexicon by modifying the code below.
    
1.1 Replace "LISTOFWORDS" with whatever you like, but create a list of words to seed (start) your lexicon
</div>

In [ ]:
LISTOFWORDS = []

<div class="alert alert-warning"><strong></strong>
1.2 Now pick a name for your lexicon (replacing "YOURCATEGORYNAMEHERE") and replace LISTOFWORDS with your list. If you'd prefer, you can use the alternatives to Reddit-based word embeddings discussed above. You can also modify the size argument.
</div>

In [ ]:
lexicon.create_category("YOURCATEGORYNAMEHERE", LISTOFWORDS, model="reddit", size=100)

<div class="alert alert-warning"><strong></strong>
1.3 Replace the name with your chosen name and run this cell
</div>

In [ ]:
df["YOURCATEGORYNAMEHERE"] = [score_category(s, "YOURCATEGORYNAMEHERE", normalize=True) for s in df["text"].values]
df["YOURCATEGORYNAMEHERE"] = standardize(df["YOURCATEGORYNAMEHERE"])

<div class="alert alert-warning"><strong>Exercise 2</strong>
    
Now you will use your lexicon to conduct a few analyses of your own. First, describe the topic you chose and a few hypotheses.


2.1 What topic did you choose, and what could it tell us about social processes and social institutions?</div>

_Your answer here_

<div class="alert alert-warning">2.2 Is text data useful for studying your topic? (It's okay if you don't think it is!)</div>

_Your answer here_

<div class="alert alert-warning"><strong>Exercise 3</strong>

Now let's think about how your topic might relate to social institutions like work, schooling, or others that might structure time use (that is, what people do at what times).

3.1 Come up with a hypothesis about when during the week redditors may be more or less likely to post about your topic to r/jobs. If you believe there's no reason to expect a relationship between the day of the week and writing about your topic, you may explain why instead. Clarify whether your hypothesis is causal or correlational (although we will not adequately _test_ causal claims at this point). You may also describe potential confounding variables.</div>

_Your answer here_


<div class="alert alert-warning">3.2 Create a plot showing variation in the frequency of posts about your topic to r/jobs over the course of the week. You may modify code from earlier in the notebook.</div>

In [ ]:
# YOUR CODE HERE

<div class="alert alert-warning">3.3 Does this plot support your hypothesis? If you have a background in statistics or want to try something new, you may conduct and refer to formal hypothesis tests. If you believe there may be confounding variables, please describe them.</div>

_Your answer here_

<div class="alert alert-warning"><strong>Exercise 4</strong>

4.1 Come up with a hypothesis about when during the _day_ redditors may be more or less likely to post about your topic to r/jobs. If you believe there's no reason to expect a relationship between time of day and writing about your topic, you may explain why instead. Clarify whether your hypothesis is causal or correlational (although we will not adequately _test_ causal claims at this point). You may also describe potential confounding variables.</div>

_Your answer here_


<div class="alert alert-warning">4.2 Create a plot showing variation in the frequency of posts about your topic to r/jobs over the course of the day. You may modify code from earlier in the notebook.</div>

In [ ]:
# YOUR CODE HERE

<div class="alert alert-warning">4.3 Does this plot support your hypothesis? If you have a background in statistics or want to try something new, you may conduct and refer to formal hypothesis tests. If you believe there may be confounding variables, please describe them.</div>

_Your answer here_

<div class="alert alert-warning"><strong>Exercise 5</strong>

5.1 Come up with a hypothesis about whether a post's karma (upvotes, the "score" field in our dataset) would be related to the extent your topic is discussed. If you believe there is no reason to expect such a relationship, you may explain that instead. Clarify whether your hypothesis is causal or correlational (although we will not adequately _test_ causal claims at this point). You may also describe potential confounding variables.</div>

_Your answer here_

<div class="alert alert-warning">5.2 Create a plot showing the relationship between a post's score (df["score"]) and your topic. You may modify code from earlier in the notebook.</div>

In [ ]:
# YOUR CODE HERE

<div class="alert alert-warning">5.3 Does this plot support your hypothesis? If you have a background in statistics or want to try something new, you may conduct and refer to formal hypothesis tests. If you believe there may be confounding variables, please describe them.</div>

_Your answer here_

<div class="alert alert-warning"><strong>Exercise 6</strong>

6.1 Come up with a hypothesis about another lexical category that should be associated (positively or negatively) with your own category. Be sure to think about social practices. What is it about how people live that makes it likely they would (or would not) write about both your topic and this other topic in the same posts to r/jobs? Clarify whether your hypothesis is causal or correlational (although we will not adequately _test_ causal claims at this point). You may also describe potential confounding variables.</div>

_Your answer here_

<div class="alert alert-warning">6.2 Create a plot showing the relationship between your category and the category you chose. You may modify code from earlier in the notebook.</div>

In [ ]:
# YOUR CODE HERE

<div class="alert alert-warning">6.3 Does this plot support your hypothesis? If you have a background in statistics or want to try something new, you may conduct and refer to formal hypothesis tests. If you believe there may be confounding variables, please describe them.</div>

_Your answer here_